In [ ]:
#r "nuget:Microsoft.ML,1.4.0-preview"
#r "nuget:Microsoft.ML.AutoML"
#r "nuget:Microsoft.Data.DataFrame,1.0.0-e190910-1"

In [ ]:
using Microsoft.Data;
using XPlot.Plotly;

In [ ]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.RowCount); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

In [ ]:
var housingData = DataFrame.ReadCsv(@"F:\LinuxShare\housing.csv");
housingData


In [ ]:
housingData.Description()

In [ ]:
Chart.Plot(
    new Graph.Histogram()
    {
        x = housingData["median_house_value"],
        nbinsx = 20
    }
)


In [ ]:
var chart = Chart.Plot(
    new Graph.Scatter()
    {
        x = housingData["longitude"],
        y = housingData["latitude"],
        mode = "markers",
        marker = new Graph.Marker()
        {
            color = housingData["median_house_value"],
            colorscale = "Jet"
        }
    }
);

chart.Width = 600;
chart.Height = 600;
display(chart);

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.AutoML;

In [ ]:
%%time

var mlContext = new MLContext();

var experiment = mlContext.Auto().CreateRegressionExperiment(maxExperimentTimeInSeconds: 15);
var result = experiment.Execute(housingData, labelColumnName:"median_house_value");

In [ ]:
var scatters = result.RunDetails.GroupBy(
    r => r.TrainerName,
    (name, details) => new Graph.Scatter()
    {
        name = name,
        x = details.Select(r => r.RuntimeInSeconds),
        y = details.Select(r => r.ValidationMetrics.MeanAbsoluteError),
        mode = "markers",
        marker = new Graph.Marker() { size = 12 }
    });

var chart = Chart.Plot(scatters);
chart.WithXTitle("Training Time");
chart.WithYTitle("Error");
display(chart);

Console.WriteLine($"Best Trainer:{result.BestRun.TrainerName}");

In [ ]:
var housing_test = DataFrame.ReadCsv(@"F:\LinuxShare\housing_test.csv");

var testResults = result.BestRun.Model.Transform(housing_test);

var trueValues = testResults.GetColumn<float>("median_house_value");
var predictedValues = testResults.GetColumn<float>("Score");

var predictedVsTrue = new Graph.Scatter()
{
    x = trueValues,
    y = predictedValues,
    mode = "markers",
};

var maximumValue = Math.Max(trueValues.Max(), predictedValues.Max());

var perfectLine = new Graph.Scatter()
{
    x = new[] {0, maximumValue},
    y = new[] {0, maximumValue},
    mode = "lines",
};

var chart = Chart.Plot(new[] {predictedVsTrue, perfectLine });
chart.WithXTitle("True Values");
chart.WithYTitle("Predicted Values");
chart.WithLegend(false);
chart.Width = 600;
chart.Height = 600;
display(chart);